In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import os

ruta = "modelo/data/output/resultados.xlsx"

output_dir = "graficos_resultados"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print(f"Cargando datos desde: {ruta}")
print(f"Guardando gráficos en: {output_dir}/")

df_gen = pd.read_excel(ruta, sheet_name="Generadores")
df_scc = pd.read_excel(ruta, sheet_name="Cortocircuito")
df_freq = pd.read_excel(ruta, sheet_name="Frecuencia")

print("Datos cargados exitosamente.")

# --- Gráfico 1: Despacho de Generación (Stacked Bar) ---
print("Generando Gráfico 1: Despacho de Generación...")
df_gen['generator'] = 'Gen ' + df_gen['generator'].astype(str)
gen_pivot = df_gen.pivot_table(index='time', columns='generator', values='p', aggfunc='sum')
gen_pivot = gen_pivot.fillna(0)
    
plt.figure(figsize=(14, 7))
ax = gen_pivot.plot(kind='bar', stacked=True, figsize=(14, 7), width=0.8)
ax.set_title('Despacho de Generación por Hora', fontsize=16)
ax.set_xlabel('Hora (time)', fontsize=12)
ax.set_ylabel('Potencia (p) [MW]', fontsize=12)
ax.legend(title='Generador', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.xticks(rotation=45)
ax.grid(axis='y', linestyle='--', alpha=0.7)
    
if gen_pivot.index.min() == 1:
    ax.set_xticks(range(0, 24))
    ax.set_xticklabels(range(1, 25))
else:
    ax.set_xticklabels(gen_pivot.index)
    plt.tight_layout(rect=[0, 0, 0.88, 1])
plot_path_gen = os.path.join(output_dir, "despacho_generacion.png")
plt.savefig(plot_path_gen)
plt.close()
print(f"Gráfico 1 guardado en: {plot_path_gen}")

# --- Gráfico 2: Validación de la Restricción SCC (Corregido) ---
print("Generando Gráfico 2: Validación de la Restricción SCC...")

try:
    # --- INICIO DE LA MODIFICACIÓN ---
    # El gráfico original era engañoso. Ahora comparamos
    # la 'I_total' (SCC real) con la 'I_limit_real' (límite real).

    # 1. Verificar que la columna 'I_limit_real' exista
    if 'I_limit_real' not in df_scc.columns:
        print("Error: La columna 'I_limit_real' no se encontró en 'resultados.xlsx - Cortocircuito'.")
        print("Por favor, asegúrate de estar corriendo la última versión de 'main.jl' (la que te pasé) que exporta esta columna.")
        raise Exception("Columna 'I_limit_real' faltante.")

    # 2. Para cada hora, encontramos el bus que tiene la SCC MÍNIMA.
    #    idxmin() nos da el índice de la fila (bus) con el peor 'I_total'
    idx_worst_bus_per_hour = df_scc.groupby('time')['I_total'].idxmin()
    df_worst_case = df_scc.loc[idx_worst_bus_per_hour]

    # 3. Ahora df_worst_case tiene la fila completa (I_total, I_limit_real)
    #    del bus más débil en cada hora.

    plt.figure(figsize=(14, 7))
    
    # 4. Graficar la SCC mínima real (Línea Azul)
    ax = df_worst_case.plot(x='time', y='I_total', marker='o', label='SCC Mínima del Sistema ($I_{total}$)', color='dodgerblue', linewidth=2, figsize=(14,7))
    
    # 5. Graficar el LÍMITE REAL en ese bus (Línea Roja)
    #    Usamos 'ax=ax' para que grafique en el mismo eje
    df_worst_case.plot(x='time', y='I_limit_real', marker='x', linestyle='--', label='Límite Real Requerido ($I_{limit\_real}$)', color='red', linewidth=2, ax=ax)
    
    ax.set_title('Validación de Restricción SCC: Peor Bus del Sistema por Hora', fontsize=16)
    ax.set_xlabel('Hora (time)', fontsize=12)
    ax.set_ylabel('SCC [p.u.]', fontsize=12)
    ax.legend(fontsize=12)
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # 6. (Opcional) Rellenar el área donde se cumple la restricción
    ax.fill_between(
        df_worst_case['time'], 
        df_worst_case['I_total'], 
        df_worst_case['I_limit_real'], 
        where=(df_worst_case['I_total'] >= df_worst_case['I_limit_real']), 
        color='green', 
        alpha=0.2, 
        label='Margen de Cumplimiento'
    )
    
    times = df_worst_case['time']
    ax.set_xticks(ticks=times[::1])
    ax.set_xticklabels([str(t) for t in times[::1]])
    ax.set_xlim(times.min(), times.max())
    
    plt.tight_layout()
    
    plot_path_scc = os.path.join(output_dir, "scc_minima_sistema.png")
    plt.savefig(plot_path_scc)
    plt.close()
    print(f"Gráfico 2 guardado en: {plot_path_scc}")

except Exception as e:
    print(f"Error al generar Gráfico 2: {e}")

# --- Gráfico 3: Inercia Equivalente del Sistema ---
# (Este gráfico está bien, no se necesita cambiar)
print("Generando Gráfico 3: Inercia del Sistema...")
plt.figure(figsize=(14, 7))
ax = df_freq.plot(x='time', y='inertia_equiv', kind='line', marker='s', label='Inercia Equivalente', color='green', linewidth=2, figsize=(14, 7))
ax.set_title('Inercia Equivalente del Sistema por Hora', fontsize=16)
ax.set_xlabel('Hora (time)', fontsize=12)
ax.set_ylabel('Inercia (inertia_equiv) [MWs]', fontsize=12)
ax.legend(fontsize=12)
ax.grid(axis='y', linestyle='--', alpha=0.7)
times = df_freq['time']
ax.set_xticks(ticks=times[::1])
ax.set_xticklabels([str(t) for t in times[::1]])
ax.set_xlim(times.min(), times.max())
plt.tight_layout()
plot_path_freq = os.path.join(output_dir, "inercia_sistema.png")
plt.savefig(plot_path_freq)
plt.close()
print(f"Gráfico 3 guardado en: {plot_path_freq}")

# --- Gráfico 4: RoCoF del Sistema ---
print("Generando Gráfico 4: RoCoF del Sistema...")
rocof_limit = 0.5
plt.figure(figsize=(14, 7))
plt.plot(df_freq['time'], df_freq['rocof'], marker='x', linestyle='-', label='RoCoF del Sistema', color='purple', linewidth=2)
plt.axhline(y=rocof_limit, color='red', linestyle='--', linewidth=2, label=f'Límite RoCoF = {rocof_limit} Hz/s')
plt.title('RoCoF (Rate of Change of Frequency) del Sistema por Hora', fontsize=16)
plt.xlabel('Hora (time)', fontsize=12)
plt.ylabel('RoCoF [Hz/s]', fontsize=12)
plt.legend(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
times = df_freq['time']
plt.xticks(ticks=times[::1], labels=[str(t) for t in times[::1]])
plt.xlim(times.min(), times.max())
max_val = max(df_freq['rocof'].max(), rocof_limit)
plt.ylim(0, max_val * 1.1)
plt.tight_layout()
plot_path_rocof = os.path.join(output_dir, "rocof_sistema.png")
plt.savefig(plot_path_rocof)
plt.close()
print(f"Gráfico 4 guardado en: {plot_path_rocof}")

print("\n--- Proceso de graficación completado ---")

Cargando datos desde: modelo/data/output/resultados.xlsx
Guardando gráficos en: graficos_resultados/
Datos cargados exitosamente.
Generando Gráfico 1: Despacho de Generación...
Gráfico 1 guardado en: graficos_resultados\despacho_generacion.png
Generando Gráfico 2: Validación de la Restricción SCC...
Gráfico 2 guardado en: graficos_resultados\scc_minima_sistema.png
Generando Gráfico 3: Inercia del Sistema...
Gráfico 3 guardado en: graficos_resultados\inercia_sistema.png
Generando Gráfico 4: RoCoF del Sistema...
Gráfico 4 guardado en: graficos_resultados\rocof_sistema.png

--- Proceso de graficación completado ---


<Figure size 1400x700 with 0 Axes>

<Figure size 1400x700 with 0 Axes>

<Figure size 1400x700 with 0 Axes>

Mapeo de Buses